In [1]:
import cobra
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
from cobra.flux_analysis import flux_variability_analysis

In [2]:
M_xanthus = read_sbml_model("../M_xanthus_model.sbml")
M_xanthus

Name,myxo_model
Memory address,70fbe9895010
Number of metabolites,1280
Number of reactions,1367
Number of genes,1192
Number of groups,0
Objective expression,1.0*OF_BIOMASS - 1.0*OF_BIOMASS_reverse_80d2e
Compartments,"c, e"


In [2]:
E_coli = load_json_model("../E_coli_model.json")
E_coli

Name,iML1515
Memory address,7e17d7d90ad0
Number of metabolites,1877
Number of reactions,2714
Number of genes,1516
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [ ]:
print("E_coli: " + str(E_coli.compartments))
print("M_xanthus: " + str(M_xanthus.compartments)) #Both Gram-, both should have periplasm

E_coli: {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
M_xanthus: {'c': '', 'e': ''}


In [ ]:
pFBA = cobra.flux_analysis.pfba(M_xanthus)
FBA = M_xanthus.optimize()
print(FBA.objective_value, pFBA.objective_value) #Not the same result (not even close)

70.09970088263155 51953.14196186691


In [55]:
pFBA

,fluxes,reduced_costs
rxn02201_c,0.00000,135.200000
rxn00351_c,0.00000,264.400000
rxn07431_c,0.00000,2.000000
rxn00836_c,0.00000,-139.200000
rxn01094_c,0.00000,-2.000000
...,...,...
EX_lac_D_e,1000.00000,-2.000000
EX_tsul_e,0.00000,2.000000
DM_glycogenb_c,0.00000,-2.000000
DM_Biomass_c,122.49922,-2.000000


In [52]:
M_xanthus.reactions.EX_lac_D_e

Reaction identifier,EX_lac_D_e
Name,Exchange for D-Lactate [e]
Memory address,0x79446f4e6990
Stoichiometry,lac_D_e <=> D-Lactate [e] <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [4]:
E_coli.reactions.AACPS4

Reaction identifier,AACPS4
Name,Acyl-[acyl-carrier-protein] synthetase (n-C16:1)
Memory address,0x7e17d6ec4110
Stoichiometry,ACP_c + atp_c + hdcea_c --> amp_c + hdeACP_c + ppi_c Acyl carrier protein + ATP C10H12N5O13P3 + Hexadecenoate (n-C16:1) --> AMP C10H12N5O7P + Cis-hexadec-9-enoyl-[acyl-carrier protein] (n-C16:1) + Diphosphate
GPR,b2836 and b1094
Lower bound,0.0
Upper bound,51.87528627275535
